In [1]:
%cd ../src
!dir

D:\work\kaggle\credit_risk\src
 驱动器 D 中的卷是 LENOVO
 卷的序列号是 6AAD-1BCB

 D:\work\kaggle\credit_risk\src 的目录

2018/05/21  16:36    <DIR>          .
2018/05/21  16:36    <DIR>          ..
2018/05/21  17:24    <DIR>          models
2018/05/20  17:03            98,302 summary.log
2018/05/20  17:00             1,247 summary.py
2018/05/21  17:20             3,712 train.py
               3 个文件        103,261 字节
               3 个目录 116,596,887,552 可用字节


In [2]:
import os
import gc
import os.path as op
import numpy as np
import feather
import pandas as pd
import fire
import models
# import ..src.models as models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:

def cache_read(csv_file, cache_suffix=".feather", **kwargs):
	"""
	用pandas读取csv文件，查看是否具有默认的缓存文件，
	如果没有缓存文件，那么按照原始文件进行读取
	"""
	cache_file = csv_file + cache_suffix
	if op.exists(cache_file):
		data = feather.read_dataframe(cache_file)
	else:
		data = pd.read_csv(csv_file, **kwargs)
		feather.write_dataframe(data, cache_file)
	return data


In [27]:
	# data directory
	cur_dir = '.' # op.dirname(__file__)
	data_dir = op.join(cur_dir, '../data')

	# read data
	print("Processing reading raw data ...")
	train_file = op.join(data_dir, 'application_train.csv.zip')
	train = cache_read(train_file)
	test_file = op.join(data_dir, 'application_test.csv.zip')
	test = cache_read(test_file)
test.head()

Processing reading raw data ...


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:


	# bu tag
	bureau_file = op.join(data_dir, 'bureau.csv.zip')
	bureau = cache_read(bureau_file)

	# bub tag
	bureau_balance_file = op.join(data_dir, 'bureau_balance.csv.zip')
	bureau_balance = cache_read(bureau_balance_file)

	# ccb tag
	credit_card_balance_file = op.join(data_dir, 'credit_card_balance.csv.zip')
	credit_card_balance = cache_read(credit_card_balance_file)

	# itp tag
	installments_payments_file = op.join(data_dir, 'installments_payments.csv.zip')
	installments_payments = cache_read(installments_payments_file)

	# pcb tag
	pos_cash_balance_file = op.join(data_dir, 'POS_CASH_balance.csv.zip')
	pos_cash_balance = cache_read(pos_cash_balance_file)

	# pa tag
	previous_application_file = op.join(data_dir, 'previous_application.csv.zip')
	previous_app = cache_read(previous_application_file)

Processing reading raw data ...


In [ ]:
# bureau_balance[bureau_balance['SK_ID_BUREAU'] == 5715448]

# df = bureau_balance.groupby(['SK_ID_BUREAU'])['MONTHS_BALANCE'].count().sort_values()
# df
# bureau_balance[bureau_balance['SK_ID_BUREAU'] == 6052856]

# 信用局中对每次记录的历史，包括贷款记录等，我们可以统计每笔贷款的月长以及状态的数量
df1 = bureau_balance.groupby(['SK_ID_BUREAU'])['MONTHS_BALANCE'].count().sort_values().reset_index()
df2 = bureau_balance.groupby(['SK_ID_BUREAU', 'STATUS'])['STATUS'].count()
df2 = df2.unstack('STATUS').fillna(0).astype('uint16')
del df2.columns.name
df2 = df2.reset_index()
df = pd.merge(df1, df2, how='left', on=['SK_ID_BUREAU'])
df.head()

In [ ]:
bureau2 = pd.merge(bureau, df, how='left', on=['SK_ID_BUREAU'])
bureau2.isnull().sum() / bureau2.shape[0]

In [ ]:
# credit_card_balance.isnull().sum() / credit_card_balance.shape[0]
credit_card_balance = credit_card_balance.sort_values(['SK_ID_CURR','SK_ID_PREV', 'MONTHS_BALANCE'])
credit_card_balance.head()

In [ ]:
s1 = credit_card_balance.groupby('SK_ID_CURR')['SK_ID_PREV'].nunique()
s1.sort_values().tail()

In [ ]:
# credit_card_balance[credit_card_balance['SK_ID_CURR'] == 355767]
(s1 > 1).sum() / len(s1)

In [ ]:
credit_card_balance.tail()

In [ ]:
installments_payments = installments_payments.sort_values(['SK_ID_CURR', 'SK_ID_PREV'])
installments_payments.head()

In [ ]:
pos_cash_balance = pos_cash_balance.sort_values(['SK_ID_CURR', 'SK_ID_PREV', 'MONTHS_BALANCE'])
pos_cash_balance.head()

In [ ]:
pos_cash_balance[(pos_cash_balance['CNT_INSTALMENT_FUTURE'] == 0) & (pos_cash_balance['NAME_CONTRACT_STATUS'] != 'Completed')].tail()

In [ ]:
pos_cash_balance.isnull().sum() / pos_cash_balance.shape[0], pos_cash_balance.shape[0]

In [ ]:
previous_app = previous_app.sort_values(['SK_ID_CURR', 'SK_ID_PREV'])
previous_app.head()

In [5]:
n_folds = 3
opt = {'n_estimator': 200, 'early_stopping_round': 10}
clf = models.GBMClassifier(opt)

In [6]:
# 训练和测试集是独立的，且每个都是无重复的ID。
print(train['SK_ID_CURR'].nunique() == train.shape[0])
print(test['SK_ID_CURR'].nunique() == test.shape[0])
print(len(set(train['SK_ID_CURR'].tolist()) & set(test['SK_ID_CURR'].tolist())))

True
True
0


In [7]:
	y = train['TARGET']
	train.drop(['TARGET'], axis=1, inplace=True)
	train_size = train.shape[0]
	train = pd.concat([train, test])
	del test

In [8]:
train = train.fillna(-99)       # must before make category

# make category
for _f in train.columns.tolist():
    
    if train[_f].dtype == 'object':
        train[_f] = train[_f].astype('category')

In [9]:
	

	# split the data into train and test
	test = train.iloc[train_size:, :]
	train = train.iloc[:train_size, :]

	feat_selected = train.columns.tolist()[1:]

	# do stacking.
	cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=521)
	train_pred = np.zeros(train.shape[0])
	test_pred = np.zeros((test.shape[0], n_folds))
	feat_imp = pd.DataFrame(np.zeros((len(feat_selected), n_folds)))
	feat_imp['features'] = feat_selected

In [14]:
	for k, (trn_idx, val_idx) in enumerate(cv.split(train, y)):
		trn_x, trn_y = train[feat_selected].iloc[trn_idx], y.iloc[trn_idx]
		val_x, val_y = train[feat_selected].iloc[val_idx], y.iloc[val_idx]

		clf.fit(train_set=(trn_x, trn_y), valid_set=(val_x, val_y))

		train_pred[val_idx] = clf.predict_proba(val_x)
		test_pred[:, k] = clf.predict_proba(test[feat_selected])

		stat = roc_auc_score(val_y, train_pred[val_idx])
		print("K={}, AUC: {:.3f}".format(k, stat))

		# collect importance info
		feat_imp.iloc[:, k] = clf.get_feat_imp()

[1]	valid's auc: 0.67321
Training until validation scores don't improve for 10 rounds.
[2]	valid's auc: 0.708593
[3]	valid's auc: 0.712618
[4]	valid's auc: 0.722042
[5]	valid's auc: 0.720779
[6]	valid's auc: 0.721966
[7]	valid's auc: 0.724119
[8]	valid's auc: 0.724251
[9]	valid's auc: 0.727583
[10]	valid's auc: 0.727349
[11]	valid's auc: 0.729123
[12]	valid's auc: 0.728262
[13]	valid's auc: 0.727577
[14]	valid's auc: 0.727471
[15]	valid's auc: 0.728071
[16]	valid's auc: 0.729134
[17]	valid's auc: 0.731204
[18]	valid's auc: 0.730675
[19]	valid's auc: 0.730639
[20]	valid's auc: 0.73122
[21]	valid's auc: 0.730845
[22]	valid's auc: 0.731629
[23]	valid's auc: 0.731436
[24]	valid's auc: 0.731428
[25]	valid's auc: 0.731556
[26]	valid's auc: 0.732376
[27]	valid's auc: 0.732361
[28]	valid's auc: 0.732281
[29]	valid's auc: 0.73355
[30]	valid's auc: 0.733272
[31]	valid's auc: 0.733086
[32]	valid's auc: 0.732935
[33]	valid's auc: 0.733778
[34]	valid's auc: 0.733711
[35]	valid's auc: 0.734484
[36]	

[93]	valid's auc: 0.75018
[94]	valid's auc: 0.750441
[95]	valid's auc: 0.750676
[96]	valid's auc: 0.750731
[97]	valid's auc: 0.750908
[98]	valid's auc: 0.750992
[99]	valid's auc: 0.751086
[100]	valid's auc: 0.75115
Did not meet early stopping. Best iteration is:
[100]	valid's auc: 0.75115
K=2, AUC: 0.751


In [17]:
print(train_pred.shape)
print(test_pred.shape)
print(feat_imp.shape)

(307511,)
(48744, 3)
(120, 4)


In [18]:
feat_imp.head()

,0,1,2,features
0,28,25,34,NAME_CONTRACT_TYPE
1,82,82,82,CODE_GENDER
2,24,36,33,FLAG_OWN_CAR
3,3,2,1,FLAG_OWN_REALTY
4,0,1,4,CNT_CHILDREN


In [25]:
feat_imp['imp_mean'] = feat_imp.iloc[:, :n_folds].mean(axis=1)
feat_imp['imp_std'] = feat_imp.iloc[:, :n_folds].std(axis=1)
feat_imp = feat_imp.iloc[:, n_folds:]
feat_imp.head()

,features,imp_mean,imp_std
0,NAME_CONTRACT_TYPE,29.000000,4.582576
1,CODE_GENDER,82.000000,0.000000
2,FLAG_OWN_CAR,31.000000,6.244998
3,FLAG_OWN_REALTY,2.000000,1.000000
4,CNT_CHILDREN,1.666667,2.081666


In [29]:
np.mean(test_pred, axis=1).shape, test_pred.shape

((48744,), (48744, 3))